# Store images in sqlite - some variants

Test a few ways how to store numpy arrays of images into a sqlite database.

In [1]:
import time

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, sessionmaker

from sqlalchemy import Boolean, Column, Integer, String, JSON, Float, Table, BLOB, TypeDecorator


class MyImage(TypeDecorator):
    impl = BLOB

    def __init__(self):
        super().__init__()

    def process_literal_param(self, value, dialect):
        # convert python numpy array to sql binary blob
        return value.tobytes() if value is not None else None

    process_bind_param = process_literal_param

    def process_result_value(self, value, dialect):
        # convert sql binary blob to python numpy array
        return np.frombuffer(value) if value is not None else None


SQLALCHEMY_DATABASE_URL = 'sqlite:///./test.db'

engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={'check_same_thread': False}
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


class TableBytes(Base):
    __tablename__ = 'TableBytes'

    id = Column(Integer, primary_key=True, index=True)

    image_1 = Column(BLOB)
    image_2 = Column(BLOB)
    image_3 = Column(BLOB)
    
    
class TableJSON(Base):
    __tablename__ = 'TableJSON'

    id = Column(Integer, primary_key=True, index=True)

    image_1 = Column(JSON)
    image_2 = Column(JSON)
    image_3 = Column(JSON)
    
    
class TableMyImage(Base):
    __tablename__ = 'TableMyImage'

    id = Column(Integer, primary_key=True, index=True)

    image_1 = Column(MyImage)
    image_2 = Column(MyImage)
    image_3 = Column(MyImage)


Base.metadata.create_all(bind=engine)

In [2]:
db = SessionLocal()

insert = {
    'bytes': [],
    'json': [],
    'myimage': [],
}

nr_runs = 100

for i in range(nr_runs):
    t1 = np.random.randint(0, 256, size=(256, 256, 3))
    t2 = np.random.randint(0, 256, size=(256, 256, 3))
    t3 = np.random.randint(0, 256, size=(256, 256, 3))

    attributes_bytes = {
        'image_1': t1.tobytes(),
        'image_2': t2.tobytes(),
        'image_3': t3.tobytes(),
    }

    attributes_json = {
        'image_1': t1.tolist(),
        'image_2': t2.tolist(),
        'image_3': t3.tolist(),
    }
    
    attributes = {
        'image_1': t1,
        'image_2': t2,
        'image_3': t3,
    }

    start = time.time()
    
    db_new_part = TableBytes(**attributes_bytes)
    db.merge(db_new_part)
    db.commit()
    
    end = time.time()
    insert['bytes'].append(end - start)
    
    start = time.time()

    db_new_part = TableJSON(**attributes_json)
    db.merge(db_new_part)
    db.commit()
    
    end = time.time()
    insert['json'].append(end - start)
    
    start = time.time()

    db_new_part = TableMyImage(**attributes)
    db.merge(db_new_part)
    db.commit()
    
    end = time.time()
    insert['myimage'].append(end - start)
    

print('Insert Bytes', np.mean(insert['bytes']) * nr_runs)
print('Insert JSON', np.mean(insert['json']) * nr_runs)
print('Insert MyImages', np.mean(insert['myimage']) * nr_runs)

Insert Bytes 26.608043670654297
Insert JSON 33.67191410064697
Insert MyImages 27.967302083969116


In [3]:
db = SessionLocal()

start = time.time()
[(np.frombuffer(x[0]), np.frombuffer(x[1]), np.frombuffer(x[2])) for x in db.query(TableBytes.image_1, TableBytes.image_2, TableBytes.image_3).all()]
end = time.time()

print('Get Bytes', end - start)

start = time.time()
db.query(TableJSON.image_1, TableJSON.image_2, TableJSON.image_3).all()
end = time.time()

print('Get JSON', end - start)

start = time.time()
db.query(TableMyImage.image_1, TableMyImage.image_2, TableMyImage.image_3).all()
end = time.time()

print('Get MyImage', end - start)

Get Bytes 0.9181034564971924
Get JSON 18.040717124938965
Get MyImage 0.5609860420227051
